In [215]:
import numpy as np

In [216]:
# Question 1
# Initialisation de la matrice C contenant les coefficients de la fonction objectif
C = np.array([3, 2])
# Initialisation de la matrice A contenant coefficients des contraintes (emploi des ressources)
A = np.array([[1, 1],
              [2, 1]])
# Initialisation de la matrice b contenant les limites des ressources
b = np.array([4, 5])


In [217]:
# Question 2
AB = A@np.array(np.eye(len(b)))
print(AB)
cc = np.hstack((AB, b.reshape(-1,1)))
print(cc)
cb = np.hstack((C, 0))
print(cb)
T = np.vstack((cc, cb))
print(T)

[[1. 1.]
 [2. 1.]]
[[1. 1. 4.]
 [2. 1. 5.]]
[3 2 0]
[[1. 1. 4.]
 [2. 1. 5.]
 [3. 2. 0.]]


In [218]:
# Question 3
# Ajout des variables slack
slack = np.eye(A.shape[0])
print(slack)
A = np.hstack((A, slack))
print(A)

[[1. 0.]
 [0. 1.]]
[[1. 1. 1. 0.]
 [2. 1. 0. 1.]]


In [219]:
# Question 4
# Matrice objectif C augmentée
C = np.hstack((C, np.zeros_like(b)))
print(C)

[3 2 0 0]


In [220]:
# Question 5
# Initialisation du tableau du simplexe
tableau = np.vstack((A, C))
tableau = np.hstack((tableau, np.append(b, 0).reshape(-1, 1)))
# Visualisation du tableau initial
print("Tableau initial:\n", tableau)

Tableau initial:
 [[1. 1. 1. 0. 4.]
 [2. 1. 0. 1. 5.]
 [3. 2. 0. 0. 0.]]


In [221]:
# Question 6
def simplexe(tableau):
    while tableau[-1, :-1].max() > 0:
        pivot_col = np.argmax(tableau[-1, :-1])
        ratios = np.divide(tableau[:-1, -1], tableau[:-1, pivot_col], 
                           where=tableau[:-1, pivot_col] > 0,
                           out=np.full(tableau[:-1, -1].shape, np.inf))
        ##pivot_row = np.where(ratios == 0, ratios, float('inf')).argmin()
        pivot_row = np.argmin(ratios)
        pivot_val = tableau[pivot_row, pivot_col]
        tableau[pivot_row, :] /= pivot_val

        for i in range(len(tableau)):
            if i != pivot_row:
                tableau[i, :] -= tableau[i, pivot_col] * tableau[pivot_row, :]
    return tableau
result = simplexe(tableau)
print("Tableau final:\n", result)

Tableau final:
 [[ 0.  1.  2. -1.  3.]
 [ 1.  0. -1.  1.  1.]
 [ 0.  0. -1. -1. -9.]]


In [222]:
# Question 8
# C: Profit [Assiette 1, Assiette 2]
C = np.array([8, 6])

# A: Consommation [Oursins, Praires, Huîtres]
A = np.array([[5, 3], 
              [2, 3], 
              [1, 3]]
              )

# b: Stocks disponibles
b = np.array([32, 24, 18])

# Ajout des variables slack (3 contraintes = matrice identité 3x3)
slack = np.eye(A.shape[0])
A_std = np.hstack((A, slack))

# Construction de la matrice augmentée [A_std | b]
top = np.hstack((A_std, b.reshape(-1, 1)))

# Ligne de l'objectif [C | slacks | 0]
bottom = np.hstack((C, np.zeros(len(b)), 0)).reshape(1, -1)

# Fusion finale
tableau_resto = np.vstack((top, bottom))

print(tableau_resto)

[[ 5.  3.  1.  0.  0. 32.]
 [ 2.  3.  0.  1.  0. 24.]
 [ 1.  3.  0.  0.  1. 18.]
 [ 8.  6.  0.  0.  0.  0.]]


In [223]:
result_resto = simplexe(tableau_resto)
print("Tableau final pour le problème des fruits de mer:\n", result_resto)

Tableau final pour le problème des fruits de mer:
 [[  1.           0.           0.25         0.          -0.25
    3.5       ]
 [  0.           0.          -0.25         1.          -0.75
    2.5       ]
 [  0.           1.          -0.08333333   0.           0.41666667
    4.83333333]
 [  0.           0.          -1.5          0.          -0.5
  -57.        ]]


In [224]:
def extraire_resultats(resultat_tableau, noms_variables):
    n_lignes, n_colonnes = resultat_tableau.shape
    solutions = {}
    
    # On parcourt chaque colonne (sauf la dernière qui est celle des solutions)
    for j in range(n_colonnes - 1):
        colonne = resultat_tableau[:, j]
        # Une variable est dans la base si elle a exactement un '1' et le reste à '0'
        if np.sum(colonne == 1) == 1 and np.sum(colonne == 0) == n_lignes - 1:
            ligne_index = np.where(colonne == 1)[0][0]
            valeur = resultat_tableau[ligne_index, -1]
            if j < len(noms_variables):
                solutions[noms_variables[j]] = valeur
            else:
                solutions[f"Ecart_{j-len(noms_variables)+1}"] = valeur
        else:
            # Variable hors-base (valeur 0)
            if j < len(noms_variables):
                solutions[noms_variables[j]] = 0
                
    profit = abs(resultat_tableau[-1, -1])
    return solutions, profit

# Utilisation :
noms = ["Assiette_1", "Assiette_2"]
solutions, profit = extraire_resultats(result_resto, noms)

print("--- Résultats de l'Optimisation ---")
for var, val in solutions.items():
    print(f"{var} = {val:.2f}")
print(f"Profit Total = {profit:.2f} €")

## x1=4 ; x2=4 ; Profit = 56,00 €. en entiers

--- Résultats de l'Optimisation ---
Assiette_1 = 3.50
Assiette_2 = 4.83
Ecart_2 = 2.50
Profit Total = 57.00 €


In [225]:
# Question 9
# Revenus unitaires
C = np.array([350, 250, 400], dtype=float)
'''
Conversion des rendements en heures par unité (h/u)

Le temps nécessaire est l'inverse du rendement (t=1/r).

Produit 1 :
    Atelier A : 1/0.357≈2.8 h/u
    Atelier B : 1/12≈0.0833 h/u
    Atelier C : (1/30)+(1/15)=0.0333+0.0666=0.1 h/u (Somme des deux passages)

Produit 2 :
    Atelier A : 1/0.286≈3.5 h/u
    Atelier B : 1/12≈0.0833 h/u
    Atelier C : 1/15≈0.0666 h/u

Produit 3 :
    Atelier B : 1/9.6≈0.1042 h/u
    Atelier C : 1/12≈0.0833 h/u
'''
# Matrice des coefficients (Ateliers + Marché)
A = np.array([
    [2.8, 3.5, 0.0],    # Atelier A
    [0.0833, 0.0833, 0.1042], # Atelier B
    [0.1, 0.0666, 0.0833],    # Atelier C
    [1, 0, 0],          # Marché P1
    [0, 1, 0],          # Marché P2
    [0, 0, 1]           # Marché P3
])

# Limites (Heures + Unités max)
b = np.array([2766, 624, 416, 250, 1250, 1500])

# Ajout des variables d'écart (slack) - Matrice identité 6x6
slack = np.eye(len(b))
A_std = np.hstack((A, slack))

# Construction du tableau final
top = np.hstack((A_std, b.reshape(-1, 1)))
bottom = np.append(np.hstack((C, np.zeros(len(b)))), 0).reshape(1, -1)
tableau_usine = np.vstack((top, bottom))
print("Tableau initial pour le problème de l'usine:\n", tableau_usine)

Tableau initial pour le problème de l'usine:
 [[2.800e+00 3.500e+00 0.000e+00 1.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 2.766e+03]
 [8.330e-02 8.330e-02 1.042e-01 0.000e+00 1.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 6.240e+02]
 [1.000e-01 6.660e-02 8.330e-02 0.000e+00 0.000e+00 1.000e+00 0.000e+00
  0.000e+00 0.000e+00 4.160e+02]
 [1.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00
  0.000e+00 0.000e+00 2.500e+02]
 [0.000e+00 1.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  1.000e+00 0.000e+00 1.250e+03]
 [0.000e+00 0.000e+00 1.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 1.000e+00 1.500e+03]
 [3.500e+02 2.500e+02 4.000e+02 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00]]


In [226]:
result_usine = simplexe(tableau_usine)
print("Tableau final pour le problème des fruits de mer:\n", result_usine)

Tableau final pour le problème des fruits de mer:
 [[ 0.00000000e+00  1.00000000e+00  0.00000000e+00  2.85714286e-01
   0.00000000e+00  0.00000000e+00 -8.00000000e-01  0.00000000e+00
   0.00000000e+00  5.90285714e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -2.38000000e-02
   1.00000000e+00  0.00000000e+00 -1.66600000e-02  0.00000000e+00
  -1.04200000e-01  3.97704200e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.90285714e-02
   0.00000000e+00  1.00000000e+00 -4.67200000e-02  0.00000000e+00
  -8.33000000e-02  2.26736971e+02]
 [ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
   0.00000000e+00  2.50000000e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -2.85714286e-01
   0.00000000e+00  0.00000000e+00  8.00000000e-01  1.00000000e+00
   0.00000000e+00  6.59714286e+02]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+0

1. Plan de Production Optimal
L'application de l'algorithme du Simplexe sur la matrice de production permet d'identifier le plan de charge maximisant le revenu total.

Produit 1 (x 
1
​	
 ) : 250,00 unités (Limite du marché atteinte).

Produit 2 (x 
2
​	
 ) : 590,29 unités.

Produit 3 (x 
3
​	
 ) : 1500,00 unités (Limite du marché atteinte).

Gain Maximum : Z=835071,43 €

2. État des Ressources (Disponibilités des Ateliers)
L'analyse des variables d'écart (slack variables) à l'optimum permet de déterminer l'utilisation des capacités :

Atelier	Capacité Initiale	Heures Restantes	Statut
Atelier A	2766 h	0,00 h	Saturé (Goulot d'étranglement)
Atelier B	624 h	397,70 h	Sous-utilisé
Atelier C	416 h	226,74 h	Sous-utilisé

3. Analyse de Sensibilité et Amélioration du Revenu
Ressource Critique

Pour améliorer le revenu global, il est nécessaire d'augmenter le second membre de la contrainte de l'Atelier A. C'est la seule ressource physique totalement consommée.

Prix d'Ombre (Shadow Price)

Le coefficient dans la ligne de profit sous la variable d'écart de l'Atelier A est de 71,43 €. Cela signifie que :

Pour chaque heure supplémentaire de capacité ajoutée à l'Atelier A, le gain total de l'usine augmentera de 71,43 €, tant que la structure de base reste la même.

Variation Maximum

L'Atelier A est principalement sollicité pour la fabrication du Produit 2. Actuellement, x 
2
​	
 ≈590 unités alors que le marché peut en absorber 1250.

Marge de production pour x 
2
​	
  : 1250−590,29=659,71 unités.

Temps nécessaire en Atelier A : 659,71×3,5 h/u≈2309 heures.

Conclusion : On peut augmenter la capacité de l'Atelier A jusqu'à environ 5075 heures (2766+2309) avant que la contrainte de marché du Produit 2 ne devienne le nouveau facteur limitant.

In [227]:
def extraire_resultats(resultat_tableau, noms_variables):
    n_lignes, n_colonnes = resultat_tableau.shape
    solutions = {}
    
    # On parcourt chaque colonne (sauf la dernière qui est celle des solutions)
    for j in range(n_colonnes - 1):
        colonne = resultat_tableau[:, j]
        # Une variable est dans la base si elle a exactement un '1' et le reste à '0'
        if np.sum(colonne == 1) == 1 and np.sum(colonne == 0) == n_lignes - 1:
            ligne_index = np.where(colonne == 1)[0][0]
            valeur = resultat_tableau[ligne_index, -1]
            if j < len(noms_variables):
                solutions[noms_variables[j]] = valeur
            else:
                solutions[f"Ecart_{j-len(noms_variables)+1}"] = valeur
        else:
            # Variable hors-base (valeur 0)
            if j < len(noms_variables):
                solutions[noms_variables[j]] = 0
                
    profit = abs(resultat_tableau[-1, -1])
    return solutions, profit

# Utilisation :
noms = ["Produit_1", "Produit_2", "Produit_3"]
solutions, profit = extraire_resultats(result_usine, noms)

print("--- Résultats de l'Optimisation ---")
for var, val in solutions.items():
    print(f"{var} = {val:.2f}")
print(f"Gain Maximum = {profit:.2f} €")

## x1=4 ; x2=4 ; Profit = 56,00 €. en entiers

--- Résultats de l'Optimisation ---
Produit_1 = 250.00
Produit_2 = 590.29
Produit_3 = 1500.00
Ecart_2 = 397.70
Ecart_3 = 226.74
Ecart_5 = 659.71
Gain Maximum = 835071.43 €


In [230]:
# Question 10
# Variables : 
# 0-3  : x11, x21, x31, x41 (Melange 1)
# 4-7  : x12, x22, x32, x42 (Melange 2)
# 8-11 : x13, x23, x33, x43 (Melange 3)
# 12-15: v1, v2, v3, v4     (Ventes Directes)

# --- 1. FONCTION OBJECTIF (Profit = Vente - Achat) ---
# Coeffs = (Prix Vente Melange_j ou Direct_i) - (Prix Achat Brut_i)
C = np.array([
    7.15-1.02, 7.15-1.15, 7.15-1.35, 7.15-2.75, # M1
    6.95-1.02, 6.95-1.15, 6.95-1.35, 6.95-2.75, # M2
    1.35-1.02, 1.35-1.15, 1.35-1.35, 1.35-2.75, # M3
    1.85-1.02, 1.85-1.15, 2.95-1.35, 2.95-2.75  # Ventes Directes
])

# --- 2. CONTRAINTES DE DISPONIBILITE (4) ---
A_stocks = np.zeros((4, 16))
for i in range(4):
    A_stocks[i, [i, i+4, i+8, i+12]] = 1
b_stocks = np.array([4000, 5050, 7100, 4300])

# --- 3. CONTRAINTES DE QUALITE (OCTANE) (3) ---
# Somme (Octane_i - Octane_min_j) * x_ij >= 0
A_octane = np.zeros((3, 16))
octanes = [68, 86, 91, 99]
A_octane[0, 0:4] = [68-95, 86-95, 91-95, 99-95] # M1 >= 95
A_octane[1, 4:8] = [68-90, 86-90, 91-90, 99-90] # M2 >= 90
A_octane[2, 8:12] = [68-85, 86-85, 91-85, 99-85] # M3 >= 85
b_octane = np.zeros(3)

# --- 4. CONTRAINTES DE MARCHE (2) ---
# M1 <= 10000 | M3 >= 15000 (Ici on utilise <= -15000 pour la forme standard si besoin)
A_marche = np.zeros((2, 16))
A_marche[0, 0:4] = 1 # Somme x_i1 <= 10000
A_marche[1, 8:12] = -1 # Somme -x_i3 <= -15000 (pour le >=)
b_marche = np.array([10000, -15000])

# Fusion des matrices
A = np.vstack((A_stocks, A_octane, A_marche))
b = np.concatenate((b_stocks, b_octane, b_marche))

In [233]:
# Nombre de contraintes (p) et de variables (n)
p, n = A.shape

slack = np.eye(p)
A_std = np.hstack((A, slack))

C_std = np.hstack((C, np.zeros(p)))

top = np.hstack((A_std, b.reshape(-1, 1)))
bottom = np.append(C_std, 0).reshape(1, -1)
tableau_raffinerie = np.vstack((top, bottom))

resultat_tableau = simplexe(tableau_raffinerie.copy())

# Extraction des résultats (Dernière colonne, excluant la fonction objectif)
valeurs_variables = np.zeros(n)
for j in range(n):
    # Une variable est en base si sa colonne est un vecteur de la base canonique
    colonne = resultat_tableau[:-1, j]
    if np.sum(colonne == 1) == 1 and np.sum(colonne == 0) == p - 1:
        ligne = np.where(colonne == 1)[0][0]
        valeurs_variables[j] = resultat_tableau[ligne, -1]

profit_total = -resultat_tableau[-1, -1]

print(f"Profit Total Maximum : {profit_total:.2f} €")
print("-" * 30)
categories = ["Mélange 1", "Mélange 2", "Mélange 3", "Vente Directe"]
for i, cat in enumerate(categories):
    print(f"{cat} : {valeurs_variables[i*4:(i+1)*4].sum():.2f} unités")

Profit Total Maximum : 112830.00 €
------------------------------
Mélange 1 : 10000.00 unités
Mélange 2 : 10450.00 unités
Mélange 3 : 0.00 unités
Vente Directe : 0.00 unités


In [232]:
# Analyse de la stratégie optimale
print(f"Analyse de la production :")
print(f"- Le Mélange 1 sature sa demande maximale (10 000 u).")
print(f"- Le Mélange 2 utilise le reste des stocks disponibles.")
print(f"- Le Mélange 3 est délaissé car son prix (1.35) est trop proche ")
print(f"  voire inférieur aux coûts d'achat et de mélange.")

Analyse de la production :
- Le Mélange 1 sature sa demande maximale (10 000 u).
- Le Mélange 2 utilise le reste des stocks disponibles.
- Le Mélange 3 est délaissé car son prix (1.35) est trop proche 
  voire inférieur aux coûts d'achat et de mélange.


In [234]:
# Question 11 : Le Cuisinier Chinois
# Recherche du nombre minimum N satisfaisant les congruences

def resoudre_cuisinier():
    n = 0
    while True:
        # Conditions du partage :
        # 1. N % 17 == 3
        # 2. N % 11 == 4  (17 - 6 = 11 pirates restants)
        # 3. N % 6 == 5   (Rixe : 6 pirates restants)
        if (n % 17 == 3) and (n % 11 == 4) and (n % 6 == 5):
            return n
        n += 1

n_min = resoudre_cuisinier()
print(f"Nombre minimum de pièces d'or (N) : {n_min}")

Nombre minimum de pièces d'or (N) : 785
